In [1]:
import os
import numpy as np
import shutil
import glob
import csv
import cv2 as cv
import nibabel as nib
import matplotlib.pyplot as plt
from genericpath import exists
from typing import Sized
from matplotlib.figure import Figure
import json
from PIL import Image
import pandas as pd
from skimage import color
from skimage import io
from scipy.signal import correlate
from scipy.stats import multivariate_normal
import pandas as pd
from scipy.stats import kurtosis, skew, entropy, median_absolute_deviation
import warnings
import scipy.optimize
import seaborn as sns
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

# Data Loading

In [276]:
df = pd.read_csv(r'D:\OneDrive\JOB\NUXTU\Casos_positivos_de_COVID-19_en_Colombia.csv')
df.head()

,fecha reporte web,ID de caso,Fecha de notificación,Código DIVIPOLA departamento,Nombre departamento,Código DIVIPOLA municipio,Nombre municipio,Edad,Unidad de medida de edad,Sexo,...,Código ISO del país,Nombre del país,Recuperado,Fecha de inicio de síntomas,Fecha de muerte,Fecha de diagnóstico,Fecha de recuperación,Tipo de recuperación,Pertenencia étnica,Nombre del grupo étnico
0,2021-03-04 00:00:00,2265685,2021-03-02 00:00:00,11,BOGOTA,11001,BOGOTA,49,1,M,...,NaN,NaN,Recuperado,2021-02-25 00:00:00,NaN,2021-03-03 00:00:00,2021-03-16 00:00:00,Tiempo,6.0,NaN
1,2021-03-04 00:00:00,2265686,2021-03-02 00:00:00,11,BOGOTA,11001,BOGOTA,49,1,M,...,NaN,NaN,Recuperado,2021-02-23 00:00:00,NaN,2021-03-03 00:00:00,2021-03-16 00:00:00,Tiempo,6.0,NaN
2,2021-03-04 00:00:00,2265687,2021-03-02 00:00:00,11,BOGOTA,11001,BOGOTA,51,1,F,...,NaN,NaN,Recuperado,2021-02-24 00:00:00,NaN,2021-03-03 00:00:00,2021-03-16 00:00:00,Tiempo,6.0,NaN
3,2021-03-04 00:00:00,2265688,2021-03-02 00:00:00,11,BOGOTA,11001,BOGOTA,51,1,F,...,NaN,NaN,Recuperado,2021-02-27 00:00:00,NaN,2021-03-03 00:00:00,2021-03-13 00:00:00,Tiempo,6.0,NaN
4,2021-03-04 00:00:00,2265689,2021-03-02 00:00:00,11,BOGOTA,11001,BOGOTA,51,1,F,...,NaN,NaN,Recuperado,2021-03-01 00:00:00,NaN,2021-03-03 00:00:00,2021-03-16 00:00:00,Tiempo,6.0,NaN


In [277]:
print(df.shape)
print(df.columns)

(6247634, 23)
Index(['fecha reporte web', 'ID de caso', 'Fecha de notificación',
       'Código DIVIPOLA departamento', 'Nombre departamento',
       'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad',
       'Unidad de medida de edad', 'Sexo', 'Tipo de contagio',
       'Ubicación del caso', 'Estado', 'Código ISO del país',
       'Nombre del país', 'Recuperado', 'Fecha de inicio de síntomas',
       'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación',
       'Tipo de recuperación', 'Pertenencia étnica',
       'Nombre del grupo étnico'],
      dtype='object')


# Exploratory Data Analysis

The following lines allow to explore columns in the dataset, their values, data type, null entries and how unique values are distributed among columns in order to check if any column can be dropped out due to it's entries. First, the label column which is the subject of prediction is explored

In [278]:
df.Estado.value_counts()

Leve         6025656
Fallecido     140603
leve           49905
Moderado         908
Grave             76
LEVE               2
Name: Estado, dtype: int64

In [279]:
df.Estado.isnull().sum()

30484

Unique labels are:
- Leve: 6075563
- Moderado: 908 entries
- Grave: 76 entries
- Fallecido: 140603 entries
- Nan: 30483 entries

Further preprocessing will require joining 'Leve', 'leve' and 'LEVE' as unique values. The above result shows that classess are unbalanced since leve have more than 6 million entries while grave have only 76. This will be an important insight during training. 

In [280]:
df[df['Estado']=='LEVE'] = 'Leve'
df[df['Estado']=='leve'] = 'Leve'

In [281]:
df.Estado.value_counts()

Leve         6075563
Fallecido     140603
Moderado         908
Grave             76
Name: Estado, dtype: int64

According to the INS, empty "Recuperado" entries are "active" entries, therefore:

In [282]:
df.Recuperado = df.Recuperado.fillna('Activo')

In [205]:
df[df['Recuperado']=='fallecido'] = 'Fallecido'

In [283]:
df.Recuperado.value_counts()

Recuperado    6003952
Fallecido      140308
Activo          53172
Leve            49907
fallecido         295
Name: Recuperado, dtype: int64

Then, useful information can be displaied of all columns in order to verify their data type and null entries

In [284]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6247634 entries, 0 to 6247633
Data columns (total 23 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   fecha reporte web             6247634 non-null  object
 1   ID de caso                    6247634 non-null  object
 2   Fecha de notificación         6247634 non-null  object
 3   Código DIVIPOLA departamento  6247634 non-null  object
 4   Nombre departamento           6247634 non-null  object
 5   Código DIVIPOLA municipio     6247634 non-null  object
 6   Nombre municipio              6247634 non-null  object
 7   Edad                          6247634 non-null  object
 8   Unidad de medida de edad      6247634 non-null  object
 9   Sexo                          6247634 non-null  object
 10  Tipo de contagio              6247634 non-null  object
 11  Ubicación del caso            6217150 non-null  object
 12  Estado                        6217150 non-

The above results reveal that four features have null or NaN data for the mayority of their entries. These are:
- Código ISO del país
- Nombre del país
- Fecha de muerte
- Nombre del grupo étinco

In order to assess if deleting those features will negtively impact our model, unique values and their value count for each is useful. To visualize the full output select "in a text editor".

In [53]:
col = 'Código ISO del país'
print(df[col].value_counts().size)
pd.options.display.max_rows = 600000
df[col].value_counts()


98


840.0     1415
484.0      384
724.0      365
76.0       276
862.0      178
591.0      154
218.0      125
214.0       99
604.0       68
152.0       66
222.0       57
250.0       44
792.0       34
32.0        26
124.0       26
380.0       24
826.0       24
276.0       23
533.0       23
630.0       22
388.0       22
320.0       15
188.0       14
818.0       13
112.0       12
51.0        10
530.0        9
192.0        9
376.0        8
212.0        6
600.0        6
784.0        6
36.0         5
780.0        5
682.0        5
558.0        5
528.0        5
300.0        5
68.0         5
850.0        4
340.0        4
858.0        4
531.0        4
1724.0       4
620.0        3
756.0        3
1249.0       3
8.0          3
4.0          3
56.0         3
643.0        3
1580.0       2
140.0        2
422.0        2
660.0        2
410.0        2
1152.0       2
670.0        2
752.0        2
360.0        2
504.0        2
566.0        1
1528.0       1
332.0        1
156.0        1
70.0         1
86.0      

In [54]:
col = 'Nombre del país'
print(df[col].value_counts().size)
df[col].value_counts()


88


ESTADOS UNIDOS DE AMÉRICA                          1412
MÉXICO                                              383
ESPAÑA                                              369
BRASIL                                              285
VENEZUELA                                           177
ECUADOR                                             126
PANAMÁ                                              126
REPÚBLICA DOMINICANA                                 98
CHILE                                                68
EL SALVADOR                                          57
PERÚ                                                 50
FRANCIA                                              45
TURQUÍA                                              34
PANAMA                                               30
ARGENTINA                                            26
CANADÁ                                               26
ITALIA                                               25
ARUBA                                           

In [285]:
df = df.drop('Nombre del país', axis = 1)
df.shape, df.columns

((6247634, 22),
 Index(['fecha reporte web', 'ID de caso', 'Fecha de notificación',
        'Código DIVIPOLA departamento', 'Nombre departamento',
        'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad',
        'Unidad de medida de edad', 'Sexo', 'Tipo de contagio',
        'Ubicación del caso', 'Estado', 'Código ISO del país', 'Recuperado',
        'Fecha de inicio de síntomas', 'Fecha de muerte',
        'Fecha de diagnóstico', 'Fecha de recuperación', 'Tipo de recuperación',
        'Pertenencia étnica', 'Nombre del grupo étnico'],
       dtype='object'))

Since "Nombre del país" and "Código ISO del país" provide the same information and the latter have more entries, "Nombre del país" can be deleted as a feature. The same is true for "Código DIVIPOLA..." of municipalities and departments and the columns with their names. Since codes are unique and names in lower and upper caps are detected as different entries, columns with names of municipalities and departments are deleted while DIVIPOLA codes are kept, since they contain the same information.

In [286]:
print(df['Código DIVIPOLA departamento'].value_counts().shape, df['Nombre departamento'].value_counts().shape)
df['Código DIVIPOLA departamento'].value_counts().sum(), df['Nombre departamento'].value_counts().sum()

(37,) (41,)


(6247634, 6247634)

In [287]:
df = df.drop('Nombre departamento', axis = 1)
df.shape, df.columns

((6247634, 21),
 Index(['fecha reporte web', 'ID de caso', 'Fecha de notificación',
        'Código DIVIPOLA departamento', 'Código DIVIPOLA municipio',
        'Nombre municipio', 'Edad', 'Unidad de medida de edad', 'Sexo',
        'Tipo de contagio', 'Ubicación del caso', 'Estado',
        'Código ISO del país', 'Recuperado', 'Fecha de inicio de síntomas',
        'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación',
        'Tipo de recuperación', 'Pertenencia étnica',
        'Nombre del grupo étnico'],
       dtype='object'))

In [288]:
print(df['Código DIVIPOLA municipio'].value_counts().shape, df['Nombre municipio'].value_counts().shape)
df['Código DIVIPOLA municipio'].value_counts().sum(), df['Nombre municipio'].value_counts().sum()

(1122,) (1054,)


(6247634, 6247634)

In [289]:
df = df.drop('Nombre municipio', axis = 1)
df.shape, df.columns

((6247634, 20),
 Index(['fecha reporte web', 'ID de caso', 'Fecha de notificación',
        'Código DIVIPOLA departamento', 'Código DIVIPOLA municipio', 'Edad',
        'Unidad de medida de edad', 'Sexo', 'Tipo de contagio',
        'Ubicación del caso', 'Estado', 'Código ISO del país', 'Recuperado',
        'Fecha de inicio de síntomas', 'Fecha de muerte',
        'Fecha de diagnóstico', 'Fecha de recuperación', 'Tipo de recuperación',
        'Pertenencia étnica', 'Nombre del grupo étnico'],
       dtype='object'))

As for 'Fecha de muerte', further feature extraction/selection analysis will provide more insights on how usefull it will be for our model.

In [290]:
col = 'Fecha de muerte'
print(df[col].value_counts().size)
df[col].value_counts()

856


Leve                   49907
2021-06-21 00:00:00      716
2021-06-24 00:00:00      694
2021-06-20 00:00:00      684
2021-06-23 00:00:00      682
2021-06-29 00:00:00      682
2021-06-14 00:00:00      679
2021-06-19 00:00:00      674
2021-06-17 00:00:00      674
2021-06-13 00:00:00      673
2021-06-22 00:00:00      672
2021-06-16 00:00:00      666
2021-06-25 00:00:00      666
2021-06-09 00:00:00      665
2021-06-12 00:00:00      664
2021-06-28 00:00:00      662
2021-06-15 00:00:00      662
2021-06-10 00:00:00      661
2021-06-11 00:00:00      659
2021-07-03 00:00:00      647
2021-06-30 00:00:00      640
2021-07-06 00:00:00      635
2021-07-01 00:00:00      626
2021-05-28 00:00:00      624
2021-06-05 00:00:00      623
2021-07-05 00:00:00      623
2021-05-31 00:00:00      622
2021-07-02 00:00:00      621
2021-06-18 00:00:00      620
2021-06-07 00:00:00      618
2021-07-04 00:00:00      615
2021-06-06 00:00:00      611
2021-06-04 00:00:00      611
2021-06-08 00:00:00      607
2021-07-07 00:

An inspection through the unique entries in "Nombre del grupo étnico", some misleading information was found since some entries are cities, number, dates, neighbours, but not ethnical groups, e.g., entries like "LA PAZ", "13 DE MAYO", etc. Therefore, this columns will be droped as feature for the models.

In [291]:
col = 'Nombre del grupo étnico'
print(df[col].value_counts().size)
df[col].value_counts()

1854


Leve                                                                                   49907
Por definir                                                                            17191
Sin Comunidad                                                                          11259
ZENU                                                                                    9151
PIJAO                                                                                   4056
PASTO                                                                                   2740
WAYUU                                                                                   2704
MOKANA                                                                                  2290
KANKUAMO                                                                                1965
MUISCA                                                                                  1684
NASA_PAÉZ                                                             

In [292]:
df = df.drop('Nombre del grupo étnico', axis = 1)
df.shape, df.columns

((6247634, 19),
 Index(['fecha reporte web', 'ID de caso', 'Fecha de notificación',
        'Código DIVIPOLA departamento', 'Código DIVIPOLA municipio', 'Edad',
        'Unidad de medida de edad', 'Sexo', 'Tipo de contagio',
        'Ubicación del caso', 'Estado', 'Código ISO del país', 'Recuperado',
        'Fecha de inicio de síntomas', 'Fecha de muerte',
        'Fecha de diagnóstico', 'Fecha de recuperación', 'Tipo de recuperación',
        'Pertenencia étnica'],
       dtype='object'))

Checking for useful patterns in other columns with value counts, 49907 rows have 'Leve' as entry and 295 have 'Fallecido' as entry in all columns in the dataframe, therefore, these rows will be deleted from the dataset.

In [210]:
col = 'Pertenencia étnica'
print(df[col].value_counts().size)
df[col].value_counts()

6.0          5954974
5.0           135952
1.0            81837
Leve           49907
3.0              405
Fallecido        295
2.0              133
Name: Pertenencia étnica, dtype: int64

In [295]:
df = df[df['Pertenencia étnica'] != 'Leve']
df = df[df['Pertenencia étnica'] != 'Fallecido']

In [296]:
col = 'Pertenencia étnica'
print(df[col].value_counts().size)
df[col].value_counts()

5


6.0    5955259
5.0     135958
1.0      81841
3.0        405
2.0        133
Name: Pertenencia étnica, dtype: int64

In [297]:
df.columns

Index(['fecha reporte web', 'ID de caso', 'Fecha de notificación',
       'Código DIVIPOLA departamento', 'Código DIVIPOLA municipio', 'Edad',
       'Unidad de medida de edad', 'Sexo', 'Tipo de contagio',
       'Ubicación del caso', 'Estado', 'Código ISO del país', 'Recuperado',
       'Fecha de inicio de síntomas', 'Fecha de muerte',
       'Fecha de diagnóstico', 'Fecha de recuperación', 'Tipo de recuperación',
       'Pertenencia étnica'],
      dtype='object')